# ⚙️ Set Up

In [4]:
# Import Python built-in libraries
import os
import copy
import pickle
import random
import time

In [5]:
# Import pip libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm, trange

# Import torch packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

In [6]:
# Import PyG packages
import torch_geometric as pyg
import torch_geometric.data as pyg_data
from torch_geometric.typing import Adj, OptTensor
import torch_sparse

# ⚗️ Data Preprocessing

In [7]:
LOCALE = "IT"

In [8]:
with open("./raw/{}session_embedding.pkl".format(LOCALE), 'rb') as f:
    embeddings = pickle.load(f)

In [9]:
with open('./raw/{}-sessions-test.pkl'.format(LOCALE), 'rb') as f:
    sessions = pickle.load(f)

In [10]:
NUM_OF_SESSIONS = len(sessions)
NUM_OF_SESSIONS

13992

# 📦 Data Pipeline

For data ingestion, we use PyTorch's `dataloader` and PyG's `Data` class. To learn more about the `Data` class, check out the documentation [here](https://pytorch-geometric.readthedocs.io/en/latest/modules/data.html#module-torch_geometric.data).

In [11]:
class GraphDataset(pyg_data.InMemoryDataset):
    def __init__(self, root, file_name, transform=None, pre_transform=None):
        self.file_name = file_name
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [f'{self.file_name}.txt']

    @property
    def processed_file_names(self):
        return [f'{self.file_name}.pt']

    def download(self):
        pass

    def process(self):
        raw_data_file = f'{self.raw_dir}/{self.file_name}'
        with open(raw_data_file + "-sessions-test.pkl", 'rb') as f:
            sessions = pickle.load(f)


        with open(raw_data_file + "-session_embedding.pkl", 'rb') as f:
            embedding_dict = pickle.load(f)

        data_list = []

        counter = 0

        for session in sessions:
            codes, uniques = pd.factorize(session)
            senders, receivers = codes[:-1], codes[1:]

            # Build Data instance
            edge_index = torch.tensor([senders, receivers], dtype=torch.long)
            #x = torch.tensor(uniques, dtype=torch.long)
            x_new = torch.zeros((len(uniques), 100))

            item_counter = 0
            for item in uniques:
                x_new[item_counter] = torch.tensor(embedding_dict[item])
                item_counter += 1

            #y = torch.tensor([y], dtype=torch.long)

            data_list.append(pyg_data.Data(x=x_new, edge_index=edge_index))

            counter += 1

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

# 🔮 Model Design

Our gated session graph layer has two main parts: (1) message propagation to create an adjacency matrix (`self.propagate`) and (2) the GRU cell (`self.gru`). We will put these inside the `forward()` function.

We only use one layer for our `GatedSessionGraphConv` implementation for simplicity. Also, our sessions have average length < 5, so we do not need a large receptive field.

In [12]:
class GatedSessionGraphConv(pyg.nn.conv.MessagePassing):
    def __init__(self, out_channels, aggr: str = 'add', **kwargs):
        super().__init__(aggr=aggr, **kwargs)

        self.out_channels = out_channels

        self.gru = torch.nn.GRUCell(out_channels, out_channels, bias=False)

    def forward(self, x, edge_index):
        m = self.propagate(edge_index, x=x, size=None)
        x = self.gru(m, x)
        return x

    def message(self, x_j):
        return x_j

    def message_and_aggregate(self, adj_t, x):
        return matmul(adj_t, x, reduce=self.aggr)

In [13]:
class SRGNN(nn.Module):
    def __init__(self, hidden_size, n_items):
        super(SRGNN, self).__init__()
        self.hidden_size = hidden_size
        self.n_items = n_items

        self.gated = GatedSessionGraphConv(self.hidden_size)

        self.q = nn.Linear(self.hidden_size, 1)
        self.W_1 = nn.Linear(self.hidden_size, self.hidden_size, bias=False)
        self.W_2 = nn.Linear(self.hidden_size, self.hidden_size)
        self.W_3 = nn.Linear(2 * self.hidden_size, self.hidden_size, bias=False)

    def reset_parameters(self):
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)

    def forward(self, data):
        x, edge_index, batch_map = data.x, data.edge_index, data.batch

        # (0)
        #embedding = self.embedding(x).squeeze()

        # (1)-(5)
        v_i = self.gated(x, edge_index)

        # Divide nodes by session
        # For the detailed explanation of what is happening below, please refer
        # to the Medium blog post.
        sections = list(torch.bincount(batch_map).cpu())
        v_i_split = torch.split(v_i, sections)

        v_n, v_n_repeat = [], []
        for session in v_i_split:
            v_n.append(session[-1])
            v_n_repeat.append(
                session[-1].view(1, -1).repeat(session.shape[0], 1))
        v_n, v_n_repeat = torch.stack(v_n), torch.cat(v_n_repeat, dim=0)

        q1 = self.W_1(v_n_repeat)
        q2 = self.W_2(v_i)

        # (6)
        alpha = self.q(F.sigmoid(q1 + q2))
        s_g_split = torch.split(alpha * v_i, sections)

        s_g = []
        for session in s_g_split:
            s_g_session = torch.sum(session, dim=0)
            s_g.append(s_g_session)
        s_g = torch.stack(s_g)

        # (7)
        s_l = v_n
        s_h = self.W_3(torch.cat([s_l, s_g], dim=-1))
        #print("SH: ")
        #print(s_h.shape)
        #print(s_h)


        return s_h

# 🚂 Model Training

We can now start model training. The training pipeline code below was originally taken from the 2021 Fall CS224W Colab assignments and then modified to fit the model.

In [14]:
# Define the hyperparameters.
# Code taken from 2021 Fall CS224W Colab assignments.
args = {
    'batch_size': 100,
    'hidden_dim': 100,
    'epochs': 1,
    'l2_penalty': 0.00001,
    'weight_decay': 0.1,
    'step': 30,
    'lr': 0.001,
    'num_items': 466868}

class objectview(object):
    def __init__(self, d):
        self.__dict__ = d

args = objectview(args)

In [15]:
test_dataset = GraphDataset('./', LOCALE)
test_loader = pyg_data.DataLoader(test_dataset,
                                   batch_size=args.batch_size,
                                   shuffle=False,
                                   drop_last=True)

/Users/obenozgur/anaconda3/envs/notti/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [16]:
test_dataset

GraphDataset(13992)

In [17]:
test_dataset.data

/Users/obenozgur/anaconda3/envs/notti/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:157: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[42918, 100], edge_index=[2, 39422])

In [18]:
def test(loader, test_model, save_model_preds=False):
    test_model.eval()

    preds_all = torch.zeros(NUM_OF_SESSIONS, 100)
    i = 0

    for _, data in enumerate(tqdm(loader)):
        data
        with torch.no_grad():
            # max(dim=1) returns values, indices tuple; only need indices
            preds = test_model(data)
            preds_all[i*args.batch_size:(i+1)*args.batch_size,:] = preds
            i += 1

    if save_model_preds:
        np_preds = preds_all.cpu().detach().numpy()
        return np_preds


In [19]:
model = SRGNN(args.hidden_dim, args.num_items)
model.load_state_dict(torch.load("model{}".format(LOCALE), map_location=torch.device('cpu')))

<All keys matched successfully>

In [20]:
nextItems = test(test_loader, model, True)

100%|██████████| 139/139 [00:03<00:00, 36.28it/s]


# 🧪 Evaluation

In [21]:
print(nextItems.shape)

(13992, 100)


In [22]:
key_list = list(embeddings.keys())
key_list[0]

'B01LQQQWG2'

In [23]:
values_list = list(embeddings.values())
stacked_embeddings = np.stack( values_list, axis=0)
stacked_embeddings.shape

(48788, 100)

In [24]:
sim = np.dot(nextItems, stacked_embeddings.T)

In [25]:
sortedSimilarity = np.argsort(sim, axis=1)

In [26]:
sortedSimilarity = sortedSimilarity[:, -100:]

In [27]:
recommendation_lists = []

for x in range(sortedSimilarity.shape[0]):
    rec_sub_list = []
    for y in range(sortedSimilarity.shape[1]):
        rec_sub_list.append(key_list[sortedSimilarity[x,y]])
    rec_sub_list.reverse()
    recommendation_lists.append(rec_sub_list)

In [28]:
print(recommendation_lists[1905])

['B08CB4Q2YT', 'B09NQGVSPD', 'B01LQQQWG2', 'B07SSBLH13', 'B07JN5C1S3', 'B00CBE2LYQ', 'B0BDKKQGXL', 'B091V2X68B', 'B08J7PQGD7', 'B00G5YOVZA', 'B0B5YG29DQ', 'B01L01ES3M', 'B09P8KM9Z6', 'B06XFS8NFB', 'B0BDKWF3NP', 'B07DWRCL44', 'B07DGMWKK9', 'B07SP4WQVJ', 'B0036TH5VO', 'B07XKBLL8F', 'B00NPXEEC4', 'B08ZMQT158', 'B09NS737DT', 'B088P5YZYV', 'B09G9DMQ7M', 'B07NGVRYWX', 'B0B3S4QXZ3', 'B07MJTXDLR', 'B07VS2BW73', 'B01BBW9TEI', 'B091V33MX1', 'B0BDKQHP51', 'B085VKKV7L', 'B00CBE2I8K', 'B0BDKT7J4J', 'B0BDKWLP8P', 'B088N8KXD4', 'B09G98QYSD', 'B0B8T8Q3LV', 'B09QH6KD26', 'B09Q9C9QP3', 'B07J67BC5V', 'B07PNL5STG', 'B09R7ZXNDP', 'B01LSFKYWE', 'B08X6DJJ2W', 'B07PMQDWVP', 'B087HJ5ZZS', 'B07VQJ82SM', 'B0155OG8LK', 'B01HZSXZ88', 'B07PMR72ZG', 'B07SYYQ374', 'B07PHPXHQS', 'B09NQGTC8P', 'B09ZVDHR7R', 'B07ZH3DGGN', 'B0037EEK7A', 'B000H6W2GW', 'B01N5O1GL4', 'B0B4WNVXG2', 'B089KYL576', 'B074L118NQ', 'B09JLHKK8W', 'B0BBLBFKYB', 'B07GVM3ZLK', 'B01E94O9G8', 'B092323P8M', 'B0BDKBLF8C', 'B09BFG23X3', 'B01M9ERVX6', 'B08J

In [35]:
with open('./{}-recs.txt'.format(LOCALE), 'wb') as f:
    pickle.dump(recommendation_lists, f)

In [33]:
len(recommendation_lists)

13992

In [34]:
len(recommendation_lists[2])

100